In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [6]:
indep=dataset[['Age', 'EstimatedSalary',  'Gender_Male']]

In [7]:
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [8]:
dep=dataset['Purchased']

In [9]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit (X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [13]:
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)



In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'best'}: 0.8635810571647691


In [15]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [12 37]]


In [16]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.87      0.93      0.90        85
           1       0.86      0.76      0.80        49

    accuracy                           0.87       134
   macro avg       0.86      0.84      0.85       134
weighted avg       0.87      0.87      0.86       134



In [17]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.8422569027611044

In [18]:
table=pd.DataFrame.from_dict(re)

In [19]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001804,0.001448,0.005764,0.005253,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",0.843811,0.795789,0.832483,0.847020,0.808775,0.825575,0.020048,10
1,0.001414,0.001396,0.003819,0.003418,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",0.794978,0.811321,0.740173,0.868632,0.943041,0.831629,0.069140,8
2,0.002078,0.001141,0.003475,0.002057,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.804764,0.831253,0.832483,0.851527,0.924528,0.848911,0.040634,2
3,0.002584,0.001065,0.003443,0.000513,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.791752,0.811321,0.832483,0.832483,0.906166,0.834841,0.038751,6
4,0.002785,0.000522,0.003503,0.000628,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.889022,0.796284,0.832483,0.865054,0.845997,0.031208,3
5,0.001534,0.000346,0.003231,0.002031,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.843811,0.790949,0.759244,0.740173,0.867097,0.800255,0.048463,12
6,0.002520,0.000706,0.001723,0.001743,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",0.826263,0.811321,0.795256,0.869709,0.924528,0.845415,0.046678,4
7,0.003292,0.003688,0.002436,0.002636,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",0.776290,0.846714,0.852044,0.831098,0.865054,0.834240,0.030952,7
8,0.002070,0.002024,0.002074,0.003670,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.868752,0.832483,0.832483,0.886792,0.845055,0.029131,5
9,0.003162,0.002124,0.001705,0.000883,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.832255,0.888107,0.770530,0.773585,0.829124,0.818720,0.043510,11


In [23]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age:21
BMI:22
Children:1
Sex Male 0 or 1:1
Smoker Yes 0 or 1:1


In [21]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

ValueError: X has 5 features, but DecisionTreeClassifier is expecting 3 features as input.